### Sentiment Analysis with the training data and set intersection
#### Accuracy - 0.56

In [1]:
import numpy as np
import re
import pandas as pd
from nltk import wordpunct_tokenize
import string

In [3]:
pd.set_option('display.max_columns', None)

In [2]:
train_data = pd.read_csv("C:/path_to_import_from/dataset2.csv",skipinitialspace=True)
test_data = pd.read_csv("C:/path_to_import_from/dataset2_interim_eval.csv",skipinitialspace=True)

In [4]:
test_data["sentiment"] = np.nan

In [5]:
train_data.head()

,id,text,sentiment
0,0,my hair is blue.,1
1,1,i'm sweating my forthcoming trip to e3. if i ...,0
2,2,@USER i'm sorry i don't understand your last ...,0
3,3,WEBSITE2018 - my cupcakes,1
4,4,reading buyology before bedtime... great premi...,0


In [6]:
test_data.head()

,id,text,sentiment
0,0,don't wanna go home. it's all empty there. may...,NaN
1,1,going out to enjoy the sunshine again -x-,NaN
2,2,@USER good good. i'm ready for my closeup....,NaN
3,3,good morning everyone! i'm feeling blessed,NaN
4,4,@USER thank you so much i am really glad yo...,NaN


In [7]:
sentiment_data = [0,1]

#### Removed the nametags, hashtags, punctuations and smileys from the twitter feed for both train and test files

In [8]:
train_data["text"] = train_data["text"].str.replace("(@[A-Za-z0-9]+)","")
train_data["text"] = train_data["text"].str.replace("(#[A-Za-z0-9]+)","")
train_data["text"] = train_data["text"].str.replace("[{}]".format(string.punctuation), '')
train_data["text"] = train_data["text"].str.strip()
train_data["text"] = train_data["text"].str.replace("[^\w\s#@/:%.,_-]", "", flags=re.UNICODE)
train_data = train_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

test_data["text"] = test_data["text"].str.replace("(@[A-Za-z0-9]+)","")
test_data["text"] = test_data["text"].str.replace("(#[A-Za-z0-9]+)","")
test_data["text"] = test_data["text"].str.replace("[{}]".format(string.punctuation), '')
test_data["text"] = test_data["text"].str.strip()
test_data["text"] = test_data["text"].str.replace("[^\w\s#@/:%.,_-]", "", flags=re.UNICODE)
test_data = test_data.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [9]:
train_data.head()

,id,text,sentiment
0,0,my hair is blue,1
1,1,im sweating my forthcoming trip to e3 if i ca...,0
2,2,im sorry i dont understand your last comment,0
3,3,WEBSITE2018 my cupcakes,1
4,4,reading buyology before bedtime great premise ...,0


In [10]:
test_data.head()

,id,text,sentiment
0,0,dont wanna go home its all empty there maybe i...,NaN
1,1,going out to enjoy the sunshine again x,NaN
2,2,good good im ready for my closeup,NaN
3,3,good morning everyone im feeling blessed,NaN
4,4,thank you so much i am really glad you enjoye...,NaN


#### Split the words in the training file of twitter feed to list of words

In [11]:
train_data["string"] = train_data["text"].str.split(" ")

#### From the training dataset, create list of words attributed to positive(1) or negative(0) to help in the test data detection 

In [12]:
negative = pd.Series(train_data.string.where(train_data["sentiment"] == 0)).dropna()
negative = negative[:].values.tolist()
negative = [item for sublist in negative for item in sublist]

positive = pd.Series(train_data.string.where(train_data["sentiment"] == 1)).dropna()
positive = positive[:].values.tolist()
positive = [item for sublist in positive for item in sublist]

In [13]:
sentiment_language = int

#### Tokenize each twitter feed from the test data and find an intersection with the set of identified words attributed tothe sentiment in [12] command.

#### Then find the ratio of words of each sentiment in the twitter feed, then attribute the tweet to the sentiment (positive or negative) with highest ratio

In [14]:
for index, row in test_data.iterrows():
     
    f = wordpunct_tokenize(row["text"])
    words = (word.lower() for word in f)
    sentiment_ratio = {}

    sentiment = {0 : negative, 1 : positive}

    stopwordSet_negative = set(negative)
    stopwordSet_positive = set(positive)

    word_set = set(words)

    common_elements_negative = word_set.intersection(stopwordSet_negative)
    common_elements_positive = word_set.intersection(stopwordSet_positive)

    sentiment_ratio[0] = len(common_elements_negative)
    sentiment_ratio[1] = len(common_elements_positive)

    sentiment_language = str( max(sentiment_ratio, key = sentiment_ratio.get))
    #print(most_rated_language)
    test_data.sentiment.iloc[index] = sentiment_language

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


#### Test Data with identified sentiment with the help of training data

In [15]:
test_data.head()

,id,text,sentiment
0,0,dont wanna go home its all empty there maybe i...,0
1,1,going out to enjoy the sunshine again x,1
2,2,good good im ready for my closeup,0
3,3,good morning everyone im feeling blessed,1
4,4,thank you so much i am really glad you enjoye...,0


In [1]:
result = test_data[["id","sentiment"]]
result.head()

NameError: name 'test_data' is not defined

In [ ]:
result.to_csv("C:/path_to_export/sentiment_interim.csv", index = False)